# Chain of Thought Reasoning

<a href="https://colab.research.google.com/github/stevengonsalvez/LLM-dojo/blob/main/gpt-api/HLW-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
!pip install openai tiktoken python-dotenv

In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## Chain-of-Thought Prompting

In [7]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 

1. **Samsung Galaxy S23**
   - Brand: Samsung
   - Model: Galaxy S23
   - Warranty: 1 year
   - Features: Not provided
   - Description: Save up to £336 and get a pair of Galaxy Buds2 Pro worth £219
   - Data: 125 GB
   - Price monthly: £24
   - Upfront cost: £0
   - Battery: 3800 mah

2. **Apple iPhone 14 Pro Max**
   - Brand: Apple
   - Model: iPhone 14 Pro Max
   - Warranty: 3 years
   - Features: Not provided
   - Description: Save up to £288 with trade-in and get 125GB for the price of 25GB
   - Data: 125GB
   - Price monthly: £40
   - Upfront cost: Not provided
   - Battery: 4002 mah

3. **Google Pixel 6a**
   - Brand: Google
   - Model: Pixel 6a
   - Warranty: 3 years
   - Features: Not provided
   - Description: Get a Google Pixel 6a from just £24 per month and save up to £144
   - Data: 125 GB
   - Price monthly: £24
   - Upfront cost: £0
   - Battery: 3900 mah

4. **Google Pixel 7 Pro**
   - Brand: Google
   - Model: Pixel 7 Pro
   - Warranty: 3 years
   - Features: Not provided
   - Description: Save £432 with Summer Savers
   - Data: 150 GB
   - Price monthly: £50 (was £68)
   - Upfront cost: £10
   - Battery: 4355 mah

5. **Samsung Galaxy A54 5G**
   - Brand: Samsung
   - Model: Galaxy A54 5G
   - Warranty: 2 years
   - Features: 40 mp camera
   - Description: Save £432 with Summer Savers, No upfront cost
   - Data: 125GB
   - Price monthly: £33 (was £49)
   - Upfront cost: £0
   - Battery: 3900 mah


Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that mobile X is bigger than \
Laptop Y, or that mobile Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [5]:
user_message = f"""
by how much is the Galaxy A54 more expensive \
than the Google pixel 7"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between the Galaxy A54 and the Google Pixel 7.
Step 2:#### Both the Galaxy A54 and the Google Pixel 7 are available products.
Step 3:#### The user assumes that the Galaxy A54 is more expensive than the Google Pixel 7.
Step 4:#### Based on the product information, the Galaxy A54 is actually cheaper than the Google Pixel 7.
Response to user:#### The Galaxy A54 is actually cheaper than the Google Pixel 7. The Galaxy A54 is priced at £33 per month (was £49) with no upfront cost, while the Google Pixel 7 is priced at £50 per month (was £68) with a £10 upfront cost.


In [8]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking if the store sells TVs. This is a question about a specific product.

Step 2:#### TVs are not included in the list of available products. Therefore, the store does not sell TVs.

Response to user:#### I'm sorry, but we do not sell TVs. Our store specializes in mobile phones and accessories. If you have any questions about our available products, feel free to ask.


## Inner Monologue
- Since we asked the LLM to separate its reasoning steps by a delimiter, we can hide the chain-of-thought reasoning from the final output that the user sees.

In [9]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

I'm sorry, but we do not sell TVs. Our store specializes in mobile phones and accessories. If you have any questions about our available products, feel free to ask.
